In [209]:
 # kod tekrarını önleyen basit bir yöntem
class Structure:
    _fields = []
    
    def __init__(self, *args):
        for name, value in zip(self.__class__._fields, args):
            setattr(self, name, value)

class Point(Structure):
    _fields = ['x', 'y']

class User(Structure):
    _fields = ['name', 'age', 'email']

u = User('mehmet', 16, '@gmail.com'); print(u.__dict__)


{'name': 'mehmet', 'age': 16, 'email': '@gmail.com'}


In [208]:
from inspect import Signature, Parameter

sig = Signature(Parameter(pname, Parameter.POSITIONAL_OR_KEYWORD) for pname in ['name', 'age'])
def foo(*args, **kwargs):
    bound = sig.bind(*args, **kwargs)
    for k,w in bound.arguments.items():
        print(k,w)
foo('ali', 16)


name ali
age 16


In [148]:
def make_signature(names):
    return Signature(Parameter(name, Parameter.POSITIONAL_OR_KEYWORD) for name in names)

In [149]:
#decorator solution
def add_sign(*fnames):
    def decorate(cls):
        cls.__signiture__ = make_signature
        return cls
    return decorate

In [150]:
# metaclass solution
class struct_meta(type):
    def __new__(cls, name, bases, clsdict):
        clsobj = super().__new__(cls, name, bases, clsdict)
        sign = make_signature(clsobj._fields)
        setattr(clsobj, '__signature__', sign)
        return clsobj

In [151]:
class Structure(metaclass=struct_meta):
    _fields = []
    def __init__(self, *args, **kwargs):
        bound = self.__signature__.bind(*args, **kwargs)
        for name, value in bound.arguments.items():
            setattr(self, name, value)

In [156]:
class Point(Structure):
    _fields = ['x', 'y']

class User(Structure):
    _fields = ['name', 'passwd', 'email']
    def hi(self):
        locals()

In [157]:
u = User('mehmet', 'asda', '@gmail')

In [189]:
class Descriptor:
    
    def __init__(self, name=None):
        self.name = name
    
    def __set__(self, instance, value):
        print('Set', self.name, value)
        instance.__dict__[self.name] = value
    
    def __delete__(self, instance):
        print('Delete', self.name)
        del instance.__dict__[self.name]

In [190]:
class User(Structure):
    _fields = ['name', 'age', 'email']
    
    name = Descriptor('name')
    age

In [191]:
u = User('mehmet', 23, '@gmail.com')

Set name mehmet


In [194]:
class Typed(Descriptor):
    ty = object
    def __set__(self, instance, value):
        if not isinstance(value, self.ty):
            raise TypeError('Expected {}'.format(self.ty))
        return super().__set__(instance, value)

In [196]:
class Integer(Typed): ty = int
class String(Typed): ty = str
class Float(Typed): ty = float

In [197]:
class User(Structure):
    _fields = ['name', 'age', 'weight']
    
    name = String()
    age = Integer()
    weight = Float()

In [207]:
u = User('mehmet', 19, 72.4);


Set None mehmet
Set None 19
Set None 72.4


In [210]:
%pwd

'/home/fool/hobbies/metaprogramming'